# dEdx weights calculation

## Input

This notebook expects to find a txt file called "VolumesZPosition.txt". This file is coming from the _SimTracker/TrackerMaterialAnalysis_ package and more details on the setup and the procedure can be found here: 

https://twiki.cern.ch/twiki/bin/view/Sandbox/HGCalMaterialBudget

Just for the record we correct for the Materials with spaces in their name: 

sed -i -e 's/M_NEMA\ FR4\ plate/M_NEMA_FR4_plate/g' VolumesZPosition.txt

In [1]:
inputfile = "VolumesZPosition.txt"

## Some imports

In [2]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pylab as plt
import seaborn as sns

## Materials Properties

### Materials radiation lengths from Chris, Geant (in mm)

In [3]:
#In mm
#-------- 
FromChrisMatXo = OrderedDict()
FromChrisMatXo['Copper'] = 14.36
FromChrisMatXo['H_Scintillator'] = 425.4
FromChrisMatXo['Cables'] = 0.
FromChrisMatXo['M_NEMA_FR4_plate'] = 175.
FromChrisMatXo['Silicon'] = 93.66
FromChrisMatXo['Other'] = 0.
FromChrisMatXo['Air'] = 300000.
FromChrisMatXo['StainlessSteel'] = 17.35
FromChrisMatXo['WCu'] = 5.122
FromChrisMatXo['Lead'] = 5.612
#-------- 
FromGeantMatXo = OrderedDict()
FromGeantMatXo['Copper'] = 14.3559
FromGeantMatXo['H_Scintillator'] = 425.393
FromGeantMatXo['Cables'] = 66.722
FromGeantMatXo['M_NEMA_FR4_plate'] = 175.056
FromGeantMatXo['Silicon'] = 93.6762
FromGeantMatXo['Other'] = 0.
FromGeantMatXo['Air'] = 301522.
FromGeantMatXo['StainlessSteel'] = 17.3555
FromGeantMatXo['WCu'] = 5.1225
FromGeantMatXo['Lead'] = 5.6118
#-------- 
df = pd.DataFrame.from_dict([FromChrisMatXo, FromGeantMatXo])
df = df.transpose()
df.columns = ['FromChris', 'FromGeant']
print( '\033[1m' + '            Radiation Lengths in mm')
df

            Radiation Lengths in mm


,FromChris,FromGeant
Copper,14.360,14.3559
H_Scintillator,425.400,425.3930
Cables,0.000,66.7220
M_NEMA_FR4_plate,175.000,175.0560
Silicon,93.660,93.6762
Other,0.000,0.0000
Air,300000.000,301522.0000
StainlessSteel,17.350,17.3555
WCu,5.122,5.1225
Lead,5.612,5.6118


### Materials nuclear interaction lengths from Chris, Geant (in mm)

In [4]:
#In mm
#-------- 
MatNucIntLength = OrderedDict()
FromChrisMatNucIntLength = OrderedDict()
FromChrisMatNucIntLength['Copper'] = 155.1
FromChrisMatNucIntLength['H_Scintillator'] = 701.3
FromChrisMatNucIntLength['Cables'] = 0.
FromChrisMatNucIntLength['M_NEMA_FR4_plate'] = 484.2
FromChrisMatNucIntLength['Silicon'] = 457.5
FromChrisMatNucIntLength['Other'] = 0.
FromChrisMatNucIntLength['Air'] = 700000
FromChrisMatNucIntLength['StainlessSteel'] = 166
FromChrisMatNucIntLength['WCu'] = 119.9
FromChrisMatNucIntLength['Lead'] = 182.6
#-------- 
FromGeantMatNucIntLength = OrderedDict()
FromGeantMatNucIntLength['Copper'] = 155.88
FromGeantMatNucIntLength['H_Scintillator'] = 700.034
FromGeantMatNucIntLength['Cables'] = 393.71
FromGeantMatNucIntLength['M_NEMA_FR4_plate'] = 483.429
FromGeantMatNucIntLength['Silicon'] = 456.628
FromGeantMatNucIntLength['Other'] = 0.
FromGeantMatNucIntLength['Air'] = 704083
FromGeantMatNucIntLength['StainlessSteel'] = 166.272
FromGeantMatNucIntLength['WCu'] = 120.105
FromGeantMatNucIntLength['Lead'] = 182.472
#-------- 
df = pd.DataFrame.from_dict([FromChrisMatNucIntLength, FromGeantMatNucIntLength])
df = df.transpose()
df.columns = ['FromChris', 'FromGeant']
print( '\033[1m' + '            Nuclear interaction Lengths in mm')
df

            Nuclear interaction Lengths in mm


,FromChris,FromGeant
Copper,155.1,155.880
H_Scintillator,701.3,700.034
Cables,0.0,393.710
M_NEMA_FR4_plate,484.2,483.429
Silicon,457.5,456.628
Other,0.0,0.000
Air,700000.0,704083.000
StainlessSteel,166.0,166.272
WCu,119.9,120.105
Lead,182.6,182.472


### Materials dEdx from PDG, Chris, Geant (in MeV/mm)

In [5]:
#First in MeV gr^-1 cm^2 and at the end will convert to MeV/mm
dEdx = OrderedDict()
#In gr/cm^3 
rho = OrderedDict()
#--------
#Some elements necessary to build our materials
#Note: Whenever in PDG values (2018) are given in more than one state (e.g gas, liquid)
#we go to the materials.xml file and choose the corresponding value. 
dEdx['Fe'] = 1.451
dEdx['Mn'] = 1.428
dEdx['Cr'] = 1.456
dEdx['Ni'] = 1.468
dEdx['C']  = 1.745
dEdx['0']  = 1.801 
dEdx['H']  = 4.034
dEdx['Br'] = 1.380
dEdx['W']  = 1.145
dEdx['N']  = 1.813
dEdx['Ar']  = 1.519

#-------- 
rho['Fe'] = 7.874 
rho['Mn'] = 7.440 
rho['Cr'] = 7.180
rho['Ni'] = 8.902 
rho['C']  = 2.265 
rho['0']  = 1.43**-3 #Here we choose the materials.xml value.
rho['H']  = 0.07080
rho['Br'] = 3.103 
rho['W']  = 19.30 
rho['N']  = 0.8070  
rho['Ar'] = 1.639**-3 #Here we choose the materials.xml value. 

#--------
#Detector materials (From PDG and Geometry/CMSCommonData/data/materials.xml)
#You need a measurement of the density for a compound which we take from the same file
dEdx['Copper'] = 1.403
rho['Copper'] = 8.960

dEdx['H_Scintillator'] = 0.91512109*dEdx['C'] + 0.084878906*dEdx['H']
rho['H_Scintillator'] = 1.032

dEdx['Silicon'] = 1.664
rho['Silicon'] = 2.329

dEdx['M_NEMA_FR4_plate'] = 0.18077359*dEdx['Silicon'] + 0.4056325*dEdx['0'] + 0.27804208*dEdx['C'] + 0.068442752*dEdx['H'] + 0.067109079*dEdx['Br']
rho['M_NEMA_FR4_plate'] = 1.025

dEdx['Other'] = 0.
rho['Other'] = 0.

#dEdx['Air'] = 0.7494*dEdx['N'] + 0.2369*dEdx['0'] + 0.0129*dEdx['Ar'] + 0.0008*dEdx['H']
dEdx['Air'] = 0.
rho['Air'] = 1.214**-3

dEdx['StainlessSteel'] = 0.6996*dEdx['Fe']+0.01*dEdx['Mn']+0.19*dEdx['Cr']+0.1*dEdx['Ni']+0.0004*dEdx['C'];
rho['StainlessSteel'] = 8.02

dEdx['WCu'] = 0.75*dEdx['W']+0.25*dEdx['Copper']
rho['WCu'] = 14.979

dEdx['Lead'] = 1.122 #Pb
rho['Lead'] = 11.35  #Pb

dEdx['Cables'] = 0.586*dEdx['Copper'] + 0.259*dEdx['C'] + 0.138*dEdx['0'] + 0.017*dEdx['H']
rho['Cables'] = 2.68
#--------
# Now to the calculation. First, we will loop through the composite materials, not 
# to mess with the elements. 
composite_materials = ['Cables','StainlessSteel','M_NEMA_FR4_plate','H_Scintillator', 'Air', 'WCu']
for element in dEdx: 
    if element not in composite_materials: continue
    dEdx[element] = (rho[element] * dEdx[element]) / 10.
#And for the rest
for element in dEdx: 
    if element in composite_materials: continue
    dEdx[element] = (rho[element] * dEdx[element]) / 10.


#--------
FromChrisdEdx = OrderedDict()
FromChrisdEdx['Copper'] = 1.26
FromChrisdEdx['H_Scintillator'] = 0.395
FromChrisdEdx['Cables'] = 0.
FromChrisdEdx['M_NEMA_FR4_plate'] = 0.322
FromChrisdEdx['Silicon'] = 0.388
FromChrisdEdx['Other'] = 0.
FromChrisdEdx['Air'] = 0.
FromChrisdEdx['StainlessSteel'] = 1.14
FromChrisdEdx['WCu'] = 1.81
FromChrisdEdx['Lead'] = 1.27
#-------- 
#The range below is from commands like: 
# Always recheck columns in txt, so that array is pointing to desired column. 
#array=($(cat VolumesZPosition.txt | grep Copper | awk '{print $7}'))
#IFS=$'\n'
#Highest value
#echo "${array[*]}" | sort -nr | head -n1
#Lowest value
#echo "${array[*]}" | sort -nr | tail -n1

FromGeantdEdxWithGetDEDX = OrderedDict()
FromGeantdEdxWithGetDEDX['Copper'] = "1.14494 - 1.19191"
FromGeantdEdxWithGetDEDX['H_Scintillator'] = "0.155602 - 0.156968"
FromGeantdEdxWithGetDEDX['Cables'] = "0.334163 - 0.343928"
FromGeantdEdxWithGetDEDX['M_NEMA_FR4_plate'] = "0.200079 - 0.203959"
FromGeantdEdxWithGetDEDX['Silicon'] = "0.298729 - 0.349232"
FromGeantdEdxWithGetDEDX['Other'] = 0.
FromGeantdEdxWithGetDEDX['Air'] = "0.000162487 - 0.000200878"
FromGeantdEdxWithGetDEDX['StainlessSteel'] = "1.17868 - 1.24005"
FromGeantdEdxWithGetDEDX['WCu'] = "1.4676 - 1.54044"
FromGeantdEdxWithGetDEDX['Lead'] = "1.28276 - 1.39512"
#-------- 
#The range below is from commands like: 
# Always recheck columns in txt, so that array is pointing to desired column. 
#array=($(cat VolumesZPosition.txt | grep Copper | awk '{print $9}'))
#IFS=$'\n'
#Highest value
#echo "${array[*]}" | sort -nr | head -n1
#Lowest value
#echo "${array[*]}" | sort -nr | tail -n1
FromGeantdEdxWithComputeTotalDEDX = OrderedDict()
FromGeantdEdxWithComputeTotalDEDX['Copper'] = "1.25937 - 1.51496"
FromGeantdEdxWithComputeTotalDEDX['H_Scintillator'] = "0.212843 - 0.230636"
FromGeantdEdxWithComputeTotalDEDX['Cables'] = "0.458855 - 0.507352"
FromGeantdEdxWithComputeTotalDEDX['M_NEMA_FR4_plate'] = "0.319344 - 0.364818"
FromGeantdEdxWithComputeTotalDEDX['Silicon'] = "0.38849 - 0.442262"
FromGeantdEdxWithComputeTotalDEDX['Other'] = 0.
FromGeantdEdxWithComputeTotalDEDX['Air'] = "0.000221838 - 0.000303024"
FromGeantdEdxWithComputeTotalDEDX['StainlessSteel'] = "1.18145 - 1.40497"
FromGeantdEdxWithComputeTotalDEDX['WCu'] = "1.85741 - 2.18222"
FromGeantdEdxWithComputeTotalDEDX['Lead'] = "1.28388 - 1.52941"
#-------- 
df = pd.DataFrame.from_dict([dEdx,FromChrisdEdx, FromGeantdEdxWithGetDEDX, FromGeantdEdxWithComputeTotalDEDX])
df = df.transpose()[11:]
df.columns = ['FromPDG','FromChris', 'FromGeantWithGetDEDX', 'FromGeantWithComputeTotalDEDX']
print( '\033[1m' + '                              dEdx in MeV/mm')
df

                              dEdx in MeV/mm


,FromPDG,FromChris,FromGeantWithGetDEDX,FromGeantWithComputeTotalDEDX
Copper,1.25709,1.26,1.14494 - 1.19191,1.25937 - 1.51496
H_Scintillator,0.200135,0.395,0.155602 - 0.156968,0.212843 - 0.230636
Silicon,0.387546,0.388,0.298729 - 0.349232,0.38849 - 0.442262
M_NEMA_FR4_plate,0.193237,0.322,0.200079 - 0.203959,0.319344 - 0.364818
Other,0,0,0,0
Air,0,0,0.000162487 - 0.000200878,0.000221838 - 0.000303024
StainlessSteel,1.16574,1.14,1.17868 - 1.24005,1.18145 - 1.40497
WCu,1.81171,1.81,1.4676 - 1.54044,1.85741 - 2.18222
Lead,1.27347,1.27,1.28276 - 1.39512,1.28388 - 1.52941
Cables,0.426449,0,0.334163 - 0.343928,0.458855 - 0.507352


### Choice for radiation lengths values -> Geant values

In [6]:
# Current choice -> Geant values
#In mm
MatXo = OrderedDict()
MatXo['Copper'] = 14.3559
MatXo['H_Scintillator'] = 425.393
MatXo['Cables'] = 66.722
MatXo['M_NEMA_FR4_plate'] = 175.056
MatXo['Silicon'] = 93.6762
MatXo['Other'] = 0.
MatXo['Air'] = 301522.
MatXo['StainlessSteel'] = 17.3555
MatXo['WCu'] = 5.1225
MatXo['Lead'] = 5.6118

In [7]:
# Let's read the input file
# Mat is prepoint material, Z is post point material start, so 
# upper edge of prepoint material. Etable,Efull is the energy loss in the 
# prepoint volume with GetDEDX and ComputeTotalDEDX.  
matZ = pd.read_csv(inputfile, sep=" ", header=None, names=["Mat", "Z", "Eta", "R", "Etable", "Efull"], index_col=False)

In [8]:
# We will make a new column with the physical thickness of the volumes in mm
matZ["PhysThickInmm"] = abs(matZ["Z"].shift(-1) -  matZ["Z"])
matZ["PhysThickInmm"] = matZ["PhysThickInmm"].shift(1)

In [9]:
# We will add a column that indicates the track the relevant volume belongs to. 
# The logic is that right before the next track "PhysThickInmm" column will be 
# very large. 
matZ["trackflag"] = matZ.apply(lambda row: True if row["PhysThickInmm"] < 2000. else False ,axis=1)
matZ["tracknum"] = (( matZ["trackflag"] == False) & (matZ["trackflag"].shift() == True)).cumsum()
matZ["tracknum"] = matZ["tracknum"].shift(1)
matZ = matZ.drop('trackflag', 1)

In [10]:
# Now that we have the tracknum we will not let the last Copper volume to 
# its huge thickness due to track change. According to TDR BH back is at 5137.7 mm 
# so we put 6.0 mm for that Copper. In any case this do *not* effect the 
# dEdx weights calculation since it is after the sensitive material of the last layer. 
matZ.loc[ matZ["PhysThickInmm"] > 2000. , "PhysThickInmm" ] = 6.0

In [11]:
# The line below is when comparing with Chris which has no Cables. 
# matZ = matZ.query('Mat != "Cables"')

# Again, adding a new column with the physical thickness of the volumes in radiation lengths
matZ["PhysThickInXo"] = matZ.apply(lambda row: row["PhysThickInmm"] / MatXo[row["Mat"]],axis=1)

In [12]:
# Adding a new column with the dEdx of the material that the volumes is build
matZ["dEdx"] = matZ.apply(lambda row: dEdx[row["Mat"]],axis=1)

In [13]:
# Another column with the dEdx times thickness to help us with the calculation of the 
# final dEdx weights 
matZ["dEdxtimesdx"] = matZ["dEdx"] * matZ["PhysThickInmm"]

In [14]:
# And here a column with the cumulative sum
matZ["dEdxtimesdxCum"] = matZ.groupby('tracknum')["dEdxtimesdx"].cumsum()

In [15]:
# And here a column with the cumulative sum for Etable
matZ["EtableCum"] = matZ.groupby('tracknum')["Etable"].cumsum()

In [16]:
# And here a column with the cumulative sum for Efull
matZ["EfullCum"] = matZ.groupby('tracknum')["Efull"].cumsum()

In [17]:
# And here a column for the cumulative sum in radiation length 
matZ["PhysThickInXoCum"] = matZ.groupby('tracknum')["PhysThickInXo"].cumsum()

In [18]:
# We will add a column that indicates the layer the relevant volume belongs to. 
# The logic is that if the previous material is Silicon or Scintillator
# we change layer.
matZ["layerflag"] = matZ.apply(lambda row: False if row["Mat"] == "Silicon" or row["Mat"] == "H_Scintillator" else True ,axis=1)
matZ["layer"] = ( matZ["layerflag"] == True) & (matZ["layerflag"].shift(1) == False) 
matZ["layer"] = matZ.groupby('tracknum')["layer"].cumsum()
#The convention is that layers starts from 1
matZ["layer"] = matZ.apply(lambda row: row["layer"] + 1,axis=1)

In [19]:
# Drop auxillary columns
#We need layerflag for not counting the silicon/scintillator in the dedx. 
#matZ = matZ.drop('layerflag', 1)

In [20]:
# This was my misunderstanding. There should be 53 in the layers column since 
# after the last scintillator or silicon the index will increase, there is 
# material there. In case we want to filter them out uncomment the following. 
# Be careful! Filter chooses and not disgards!
# matZ = matZ.groupby('tracknum').filter(lambda g: ~(g['layer'] == 53.0).any()  ) 

In [21]:
#display(matZ)
#matZ[(matZ["layer"] == 1) & matZ["tracknum"] == 20]
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#matZ[(matZ["tracknum"] == 6.0)]
matZ = matZ.dropna()
matZ[ matZ['tracknum'] == 1.0]
#matZ[(matZ["Mat"] == "Air")]
#matZ[ (matZ["Z"] > 3950) & (matZ["Z"] < 4000) & (matZ["Mat"] == "Silicon") & (matZ["R"] < 900)]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["tracknum"] == 5) ]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["Z"] < 3900) & (matZ["Z"] > 0) & (matZ["PhysThickInmm"] > 0.22)]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["layer"] > 28) & (matZ["layer"] < 36)& (matZ["PhysThickInmm"] < 0.22)]
#matZ[matZ["layer"] >= 36]
#matZ[(matZ["Mat"] == "WCu") & (matZ["Z"] > 3950) & (matZ["Z"] < 4000)]

,Mat,Z,Eta,R,Etable,Efull,PhysThickInmm,tracknum,PhysThickInXo,dEdx,dEdxtimesdx,dEdxtimesdxCum,EtableCum,EfullCum,PhysThickInXoCum,layerflag,layer
1019,StainlessSteel,-3190.80,-2.26529,668.666,0.358472,0.369481,0.30,1.0,0.017286,1.165737,0.349721,0.349721,0.358472,0.369481,0.017286,True,1.0
1020,Lead,-3192.90,-2.26569,669.107,2.794680,2.871460,2.10,1.0,0.374211,1.273470,2.674287,3.024008,3.153152,3.240941,0.391497,True,1.0
1021,Copper,-3193.00,-2.26568,669.128,0.114805,0.131771,0.10,1.0,0.006966,1.257088,0.125709,3.149717,3.267957,3.372712,0.398463,True,1.0
1022,StainlessSteel,-3193.30,-2.26567,669.191,0.358480,0.369505,0.30,1.0,0.017286,1.165737,0.349721,3.499438,3.626437,3.742217,0.415748,True,1.0
1023,M_NEMA_FR4_plate,-3194.90,-2.26570,669.527,0.320465,0.526305,1.60,1.0,0.009140,0.193237,0.309180,3.808618,3.946902,4.268522,0.424888,True,1.0
1024,Air,-3196.40,-2.26570,669.841,0.000250,0.000351,1.50,1.0,0.000005,0.000000,0.000000,3.808618,3.947152,4.268873,0.424893,True,1.0
1025,M_NEMA_FR4_plate,-3198.00,-2.26557,670.177,0.320467,0.526337,1.60,1.0,0.009140,0.193237,0.309180,4.117798,4.267619,4.795210,0.434033,True,1.0
1026,Silicon,-3198.12,-2.26552,670.202,0.041900,0.050975,0.12,1.0,0.001281,0.387546,0.046505,4.164303,4.309519,4.846185,0.435314,False,1.0
1027,Silicon,-3198.30,-2.26560,670.240,0.062850,0.076462,0.18,1.0,0.001922,0.387546,0.069758,4.234062,4.372370,4.922647,0.437236,False,1.0
1028,WCu,-3199.70,-2.26708,670.534,2.076030,2.745810,1.40,1.0,0.273304,1.811710,2.536394,6.770456,6.448400,7.668457,0.710540,True,2.0


In [22]:
#matZ[matZ["Z"]>0]
#matZ[matZ["PhysThickInmm"]> 2000.]
#matZ[matZ["tracknum"] == 2 ]
#matZ[ (matZ["tracknum"] == 20) & (matZ["layer"] > 10) ]
#matZ[450:500]

In [23]:
# The dEdx weights calculation doesn't include the sensitive material. 
matZ = matZ.drop(matZ[ (matZ.Mat == "Silicon") | (matZ.Mat == "H_Scintillator") ].index)

In [24]:
# We will add a column with the dedx weights. First dedxtimesdx sum per layer 
# This is with the theoretical dedx
mdEdxtimesdxsumperlayer = matZ.groupby(['tracknum','layer'])["dEdxtimesdx"].sum()
# And now with the detailed simulation
mdEdxtimesdxsumperlayer_detailedtable = matZ.groupby(['tracknum','layer'])["Etable"].sum()
mdEdxtimesdxsumperlayer_detailedfull = matZ.groupby(['tracknum','layer'])["Efull"].sum()

#type(mdEdxtimesdxsumperlayer)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.to_frame().reset_index()
mdEdxtimesdxsumperlayer_detailedtable = mdEdxtimesdxsumperlayer_detailedtable.to_frame().reset_index()
mdEdxtimesdxsumperlayer_detailedfull = mdEdxtimesdxsumperlayer_detailedfull.to_frame().reset_index()

#mdEdxtimesdxsumperlayer
#type(mdEdxtimesdxsumperlayer)

In [25]:
#Let's put also the accumulated energy loss
mdEdxtimesdxsumperlayer["dEdxtimesdxCum"] = mdEdxtimesdxsumperlayer.groupby('tracknum')["dEdxtimesdx"].cumsum()
mdEdxtimesdxsumperlayer_detailedtable["EtableCum"] = mdEdxtimesdxsumperlayer_detailedtable.groupby('tracknum')["Etable"].cumsum()
mdEdxtimesdxsumperlayer_detailedfull["EfullCum"] = mdEdxtimesdxsumperlayer_detailedfull.groupby('tracknum')["Efull"].cumsum()

In [26]:
#Final weights
mdEdxtimesdxsumperlayer["dedxweights"] = (mdEdxtimesdxsumperlayer["dEdxtimesdx"] + mdEdxtimesdxsumperlayer["dEdxtimesdx"].shift(-1))/2
mdEdxtimesdxsumperlayer_detailedtable["dedxweights_detailedsimulationtable"] = (mdEdxtimesdxsumperlayer_detailedtable["Etable"] + mdEdxtimesdxsumperlayer_detailedtable["Etable"].shift(-1))/2
mdEdxtimesdxsumperlayer_detailedfull["dedxweights_detailedsimulationfull"] = (mdEdxtimesdxsumperlayer_detailedfull["Efull"] + mdEdxtimesdxsumperlayer_detailedfull["Efull"].shift(-1))/2

In [27]:
#Hack for the last layer
mdEdxtimesdxsumperlayer.loc[mdEdxtimesdxsumperlayer["layer"] == 52.0, "dedxweights"] = 89.786205
mdEdxtimesdxsumperlayer_detailedtable.loc[mdEdxtimesdxsumperlayer_detailedtable["layer"] == 52.0, "dedxweights_detailedsimulationtable"] = 91.800143
mdEdxtimesdxsumperlayer_detailedfull.loc[mdEdxtimesdxsumperlayer_detailedfull["layer"] == 52.0, "dedxweights_detailedsimulationfull"] = 98.971647

#Drop duplicates not needed columns
mdEdxtimesdxsumperlayer_detailedtable = mdEdxtimesdxsumperlayer_detailedtable.drop(['tracknum', 'layer'], axis=1)
mdEdxtimesdxsumperlayer_detailedfull = mdEdxtimesdxsumperlayer_detailedfull.drop(['tracknum', 'layer'], axis=1)

#mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ]
#mdEdxtimesdxsumperlayer_detailed[ mdEdxtimesdxsumperlayer_detailed['tracknum'] == 6.0  ]
mdEdxtimesdxsumperlayer = pd.concat([mdEdxtimesdxsumperlayer, mdEdxtimesdxsumperlayer_detailedtable, mdEdxtimesdxsumperlayer_detailedfull], axis=1)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.dropna()

#Adding two columns dedxtable/dedxtheory and dedxfull/dedxtheory
mdEdxtimesdxsumperlayer["EtableoverEtheory"] = mdEdxtimesdxsumperlayer["Etable"]/mdEdxtimesdxsumperlayer["dEdxtimesdx"]
mdEdxtimesdxsumperlayer["EfulloverEtheory"] = mdEdxtimesdxsumperlayer["Efull"]/mdEdxtimesdxsumperlayer["dEdxtimesdx"]
mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 1.0  ][:-1].dropna()

,tracknum,layer,dEdxtimesdx,dEdxtimesdxCum,dedxweights,Etable,EtableCum,dedxweights_detailedsimulationtable,Efull,EfullCum,dedxweights_detailedsimulationfull,EtableoverEtheory,EfulloverEtheory
105,1.0,1.0,4.117798,4.117798,8.366557,4.267619,4.267619,7.654235,4.795210,4.795210,9.097840,1.036384,1.164508
106,1.0,2.0,12.615316,16.733114,10.425456,11.040850,15.308469,9.789281,13.400470,18.195680,11.498605,0.875194,1.062238
107,1.0,3.0,8.235596,24.968710,10.425456,8.537712,23.846181,9.790181,9.596740,27.792421,11.503030,1.036684,1.165276
108,1.0,4.0,12.615316,37.584026,10.425456,11.042650,34.888831,9.791512,13.409320,41.201741,11.506499,0.875337,1.062940
109,1.0,5.0,8.235596,45.819622,10.425456,8.540373,43.429204,9.792462,9.603678,50.805418,11.511174,1.037007,1.166118
110,1.0,6.0,12.615316,58.434938,10.425456,11.044550,54.473754,9.793761,13.418670,64.224088,11.514570,0.875487,1.063681
111,1.0,7.0,8.235596,66.670534,10.425456,8.542971,63.016726,9.794736,9.610469,73.834557,11.519355,1.037323,1.166943
112,1.0,8.0,12.615316,79.285850,10.425456,11.046500,74.063226,9.796032,13.428240,87.262797,11.522762,0.875642,1.064439
113,1.0,9.0,8.235596,87.521446,10.425456,8.545564,82.608790,9.796972,9.617284,96.880081,11.527302,1.037638,1.167770
114,1.0,10.0,12.615316,100.136762,10.425456,11.048380,93.657170,9.798266,13.437320,110.317401,11.530703,0.875791,1.065159


In [28]:
mdEdxtimesdxsumperlayer[["layer","EtableoverEtheory","EfulloverEtheory"]][ mdEdxtimesdxsumperlayer['tracknum'] == 1.0  ][:-1] 

,layer,EtableoverEtheory,EfulloverEtheory
105,1.0,1.036384,1.164508
106,2.0,0.875194,1.062238
107,3.0,1.036684,1.165276
108,4.0,0.875337,1.062940
109,5.0,1.037007,1.166118
110,6.0,0.875487,1.063681
111,7.0,1.037323,1.166943
112,8.0,0.875642,1.064439
113,9.0,1.037638,1.167770
114,10.0,0.875791,1.065159


In [29]:
mdEdxtimesdxsumperlayer[["layer","dedxweights","dedxweights_detailedsimulationtable","dedxweights_detailedsimulationfull"]][ mdEdxtimesdxsumperlayer['tracknum'] == 1.0  ][:-1] 

,layer,dedxweights,dedxweights_detailedsimulationtable,dedxweights_detailedsimulationfull
105,1.0,8.366557,7.654235,9.097840
106,2.0,10.425456,9.789281,11.498605
107,3.0,10.425456,9.790181,11.503030
108,4.0,10.425456,9.791512,11.506499
109,5.0,10.425456,9.792462,11.511174
110,6.0,10.425456,9.793761,11.514570
111,7.0,10.425456,9.794736,11.519355
112,8.0,10.425456,9.796032,11.522762
113,9.0,10.425456,9.796972,11.527302
114,10.0,10.425456,9.798266,11.530703


In [30]:
mdEdxtimesdxsumperlayer[["layer","dEdxtimesdxCum","EtableCum","EfullCum"]][ mdEdxtimesdxsumperlayer['tracknum'] == 1.0  ][:-1] 

,layer,dEdxtimesdxCum,EtableCum,EfullCum
105,1.0,4.117798,4.267619,4.795210
106,2.0,16.733114,15.308469,18.195680
107,3.0,24.968710,23.846181,27.792421
108,4.0,37.584026,34.888831,41.201741
109,5.0,45.819622,43.429204,50.805418
110,6.0,58.434938,54.473754,64.224088
111,7.0,66.670534,63.016726,73.834557
112,8.0,79.285850,74.063226,87.262797
113,9.0,87.521446,82.608790,96.880081
114,10.0,100.136762,93.657170,110.317401
